<a href="https://colab.research.google.com/github/BaseKan/optimisation_workshop/blob/main/dash_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id=1jqIT0Oimj1sXUfNj071u7l4OSU93Fc_y' | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt    
!curl -L -b cookies.txt -o 'app.zip' 'https://docs.google.com/uc?export=download&id=1jqIT0Oimj1sXUfNj071u7l4OSU93Fc_y&confirm='$(<confirm.txt)
!unzip app.zip
!rm -f confirm.txt cookies.txt app.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1015      0 --:--:-- --:--:-- --:--:--  1013
100  3519  100  3519    0     0   5372      0 --:--:-- --:--:-- --:--:--  5372
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    853      0 --:--:-- --:--:-- --:--:--   853
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3519  100  3519    0     0   3763      0 --:--:-- --:--:-- --:--:--  3763
Archive:  app.zip
  inflating: custom_components.py    
  inflating: __MACOSX/._custom_components.py  
  inflating: page_jaarverbruik.py    
  inflating: __MACOSX/._page_jaarverbruik.py  
  inflating: page_tijd

In [8]:
!pip install jupyter-dash
!pip install dash-bootstrap-components

     |████████████████████████████████| 194kB 12.2MB/s 


In [12]:
import pandas as pd
import os

def load_data_from_file(datapath, filename):
    path = os.path.join(datapath,filename)
    return pd.read_csv(path).assign(jaar =  filename.split('_')[2][:4],
                                                         dso = filename.split('_')[0])

def load_data(basepath='C:/Data/Code/EnergieKaart/data', type = 'Electricity'):
    datapath = os.path.join(basepath,type)
    files = os.listdir(datapath)
    files = [filename for filename in files if int(filename.split('_')[2][:4])>= 2017]
    # first file
    file = files[0]
    df = load_data_from_file(datapath,file)

    # loop over other files
    for file in files[1:]:
        temp_df = load_data_from_file(datapath,file)
        df = pd.concat([df,temp_df])
    return df

def calculate_top_ten(df, year, dso):
    df['Verbruik'] = [int(vb /1000) for vb in df['annual_consume']]
    df = df[df['dso']==dso].filter(['Verbruik','jaar','city'])
    df = (
        df.groupby(['jaar','city'])
            .agg('sum')
            .sort_values('Verbruik',ascending=False)
            .reset_index()
    )
    df = df[df['jaar'] == str(year)].iloc[:10,1:]
    df.columns = ['Stad', 'Jaarverbruik (MWh)']
    return df

def calculate_timeseries(df_elec, df_gas, cities):
    df_elec = (
        df_elec.assign(E = df_elec['annual_consume'])
            .set_index(['city','zipcode_from','zipcode_to','jaar'])
            .filter(['E','num_connections'])
        .drop_duplicates()
    )
    df_gas = (
        df_gas.assign(G = df_gas['annual_consume'])
            .set_index(['city','zipcode_from', 'zipcode_to', 'jaar'])
            .filter(['G','dso'])
        .drop_duplicates()
    )
    df = df_gas.join(df_elec).reset_index()
    df = (
        df.assign(Jaar = df['jaar'],
        Stad = df['city'],
        Verbruik = [(df['E'][i]*3.6 + df['G'][i]*35.17) for i in range(df.shape[0])])
            .filter(['Stad','Jaar','Verbruik','num_connections'])
            .groupby(['Stad','Jaar'])
            .agg('sum')
            .reset_index()
    )
    df['Gemiddeld_verbruik'] = [df['Verbruik'][i]/df['num_connections'][i] for i in range(df.shape[0])]
    df = df[[any([stad == Stad for stad in cities]) for Stad in df['Stad']]]
    return df


In [14]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import plotly.express as px

import page_jaarverbruik
import page_tijdreeks
import time

external_stylesheets = ["https://bootswatch.com/4/flatly/bootstrap.css"]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

overhead_bar = dbc.NavbarSimple(
    children = [dbc.NavItem(dbc.NavLink("Stedelijk Jaarverbruik", href='/jaarverbruik', active="exact")),
                dbc.NavItem(dbc.NavLink("Verbruik over de jaren", href='/tijdreeks', active="exact"))],
    brand='EnergieNL v0.1.0',
    color='primary',
    dark=True
)

content = html.Div(id='page-content', children=[])

app.layout = html.Div([
    dcc.Location(id='url'),
    overhead_bar,
    content
])

@app.callback(Output('page-content','children'),
              [Input('url','pathname')])

def render_page_content(pathname):
    if pathname == "/" or pathname == '/jaarverbruik':
        return page_jaarverbruik.content

    elif pathname == '/tijdreeks':
        return page_tijdreeks.content

    else:
        return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

@app.callback(Output("dso-table", "data"),
              Output('calc-box1', 'children'),
              Input("submit-table-data", 'n_clicks'),
              State("year-box", "value"),
              State("dso-box", "value"))

def update_dso_table(n_clicks, year, dso):
    start = time.time()
    df = load_data()
    df = calculate_top_ten(df=df,year=year,dso=dso)
    end = time.time()
    rekentijd_str = str(round(end-start,3))+' seconden'
    return df.to_dict('records'), rekentijd_str

@app.callback(Output("timeseries-figure", "figure"),
              Output('calc-box2', 'children'),
              Input("submit-figure-data", 'n_clicks'),
              State("city-box", "value"))

def update_timeseries_figure(n_clicks, cities):
    start = time.time()
    elec = load_data()
    gas = load_data(type='Gas')
    df = calculate_timeseries(df_elec=elec,df_gas=gas, cities=cities)
    fig = px.line(df.sort_values('Jaar'), x='Jaar', y='Gemiddeld_verbruik', color='Stad')
    end = time.time()
    rekentijd_str = str(round(end-start,3))+' seconden'
    return fig, rekentijd_str


if __name__ == "__main__":
    app.run_server(mode='inline')


<IPython.core.display.Javascript object>